This notebook checks result from sprinkling full-chain based AmBe neutrons into AmBe runs, which is also pretty much a tutorial for this package when sprinkling events. It is expected to be running in `2024.03.1` container. 

The source of simulation instruction from full-chain simulation is here at `/project/lgrandi/yuanlq/salt/ambe_instructions/minghao_aptinput.csv`. When sprinkling, we bootstrap events from the instruction above, and add an offset so that the sprinkled AmBe neutron event shows up at 50Hz.

There will be 3 datasets in the end:
- `data`: Exactly the same as offline real data.
- `simulation`: Events reconstructed using the simulation instruction only, there is nothing else in the reconstruction process.
- `sprinkled`: Events reconstructed by mixing simulation and data. Some time it is also called `salt` and they mean the same thing. 

By comparing `simulation` and `sprinkled`, we will be able to understand the ambience interference effect in reconstruction.

Lanqing & Dacheng, Jun 29 2025

# Preparation

In [ ]:
import gc
from itertools import cycle
from tabulate import tabulate
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import saltax
import straxen
import cutax
from saltax.match.utils import *

import warnings
warnings.filterwarnings("ignore")

straxen.print_versions(("strax", "straxen", "cutax", "saltax"))

Below are the AmBe runs available at `/project/lgrandi/yuanlq/salt/ambe`.

In [ ]:
xedocs_version = "global_v14"

In [ ]:
runs = [
    "051904",
    "051905",
    "051906",
    "051907",
    "051908",
    "051909",
    "051910",
    "051911",
    "051913",
    "051914",
    "051938",
    "051939",
]

In `saltax`, we need to do slightly different context definition, by specifying more information about the simulation.

In [ ]:
st_salt = saltax.contexts.sxenonnt(
    corrections_version=xedocs_version,
    saltax_mode="salt",
    output_folder="./fuse_data",
    faxconf_version="sr0_v4",
    generator_name="ambe",
    recoil=0,
)
st_simu = saltax.contexts.sxenonnt(
    corrections_version=xedocs_version,
    saltax_mode="simu",
    output_folder="./fuse_data",
    faxconf_version="sr0_v4",
    generator_name="ambe",
    recoil=0,
)

st_salt.storage += [strax.DataDirectory("/project/lgrandi/yuanlq/salt/ambe", readonly=True)]
st_simu.storage += [strax.DataDirectory("/project/lgrandi/yuanlq/salt/ambe", readonly=True)]

In [ ]:
# Load events
(
    events_simu,  # all events (with S1+S2 topology) reconstructed in simulation dataset
    events_salt,  # all events reconstructed in sprinkled dataset
    inds_dict,  # index for matching
) = saltax.load_events(
    runs, st_salt, st_simu,
    plugins=("event_info", "cuts_basic"),
)

# Overview of simulation only

This dataset should be considered without any ambience interference.

You can see that those events are from source position `topCW7d8m`.

In [ ]:
ALL_CUTS_MINIMAL = [
    'cut_daq_veto',
    'cut_interaction_exists',
    'cut_main_is_valid_triggering_peak',
    'cut_run_boundaries',
]
AmBe_CUTS_EXCEPT_S2Pattern = ALL_CUTS_MINIMAL + [
    'cut_s1_area_fraction_top',
    'cut_s1_max_pmt',
    'cut_s1_pattern_bottom',
    'cut_s1_pattern_top',
    'cut_s1_single_scatter',
    'cut_s1_tightcoin_3fold',
    'cut_s1_width',
    'cut_s2_recon_pos_diff',
    'cut_s2_single_scatter',
    'cut_s2_width',
    'cut_cs2_area_fraction_top',
]

In [ ]:
mask_simu_only_cut = apply_cut_lists(
    events_simu, AmBe_CUTS_EXCEPT_S2Pattern  # Compromise based on waveform simulation quality
)
mask_simu_only_min = apply_cut_lists(events_simu, ALL_CUTS_MINIMAL)  # Minimal cuts

plt.figure(dpi=150)
plt.scatter(events_simu["cs1"], events_simu["cs2"], color="tab:blue", s=2, label="Before Cuts")
plt.scatter(
    events_simu[mask_simu_only_min]["cs1"],
    events_simu[mask_simu_only_min]["cs2"],
    color="tab:orange",
    s=2,
    label="After ALL_CUTS_MINIMAL",
)
plt.scatter(
    events_simu[mask_simu_only_cut]["cs1"],
    events_simu[mask_simu_only_cut]["cs2"],
    color="tab:red",
    s=2,
    label="After AmBe_CUTS_EXCEPT_S2Pattern",
)
plt.xlim(0, 100)
plt.ylim(0, 6500)
plt.xlabel("cS1 [PE]")
plt.ylabel("cS2 [PE]")
plt.title("Full-chain Simulated AmBe Neutrons")
plt.legend()
plt.show()

In [ ]:
plt.figure(dpi=150)
plt.scatter(events_simu["x"], events_simu["y"], color="tab:blue", s=2, label="Before Cuts")
plt.scatter(
    events_simu[mask_simu_only_min]["x"],
    events_simu[mask_simu_only_min]["y"],
    color="tab:orange",
    s=2,
    label="After ALL_CUTS_MINIMAL",
)
plt.scatter(
    events_simu[mask_simu_only_cut]["x"],
    events_simu[mask_simu_only_cut]["y"],
    color="tab:red",
    s=2,
    label="After AmBe_CUTS_EXCEPT_S2Pattern",
)
plt.xlim(-70, 70)
plt.ylim(-70, 70)
plt.title("Full-chain Simulated AmBe Neutrons")
plt.xlabel("x [cm]")
plt.ylabel("y [cm]")
plt.legend()
plt.show()

In [ ]:
plt.figure(dpi=150)
plt.scatter(
    events_simu["x"] ** 2 + events_simu["y"] ** 2,
    events_simu["z"],
    color="tab:blue",
    s=2,
    label="Before Cuts",
)
plt.scatter(
    events_simu[mask_simu_only_min]["x"] ** 2 + events_simu[mask_simu_only_min]["y"] ** 2,
    events_simu[mask_simu_only_min]["z"],
    color="tab:orange",
    s=2,
    label="After ALL_CUTS_MINIMAL",
)
plt.scatter(
    events_simu[mask_simu_only_cut]["x"] ** 2 + events_simu[mask_simu_only_cut]["y"] ** 2,
    events_simu[mask_simu_only_cut]["z"],
    color="tab:red",
    s=2,
    label="After AmBe_CUTS_EXCEPT_S2Pattern",
)
plt.xlim(0, 4900)
plt.ylim(-150, 0)
plt.title("Full-chain Simulated AmBe Neutrons")
plt.xlabel(r"$r^2[cm^2]$")
plt.ylabel("z[cm]")
plt.legend()
plt.show()

# Let's sprinkle!

## Analysis wrapper

Here we just define a collection of analysis visualization defined in `saltax.utils`

In [ ]:
def super_mega_analysis(
    events_salt_matched_to_simu,
    events_simu_matched_to_salt,
    cut_list=AmBe_CUTS_EXCEPT_S2Pattern,
):
    print("Cut acceptance without N-1")
    get_single_cut_acc(
        events_salt_matched_to_simu, events_simu_matched_to_salt, all_cut_list=cut_list
    )
    print("Cut acceptance with N-1")
    get_n_minus_1_cut_acc(
        events_salt_matched_to_simu, events_simu_matched_to_salt, all_cut_list=cut_list
    )

    compare_templates(
        events_salt_matched_to_simu[apply_cut_lists(events_salt_matched_to_simu, cut_list)],
        events_simu_matched_to_salt[apply_cut_lists(events_simu_matched_to_salt, cut_list)],
        n_bins=16,
        title="With Cuts Ambience Interference in SR1 AmBe HEV",
    )

    _ambe_s1_cut_eff = get_cut_eff(
        events_salt_matched_to_simu,
        all_cut_list=cut_list,
        n_bins=16,
        coord="s1_area",
        plot=True,
        indv_cut_type="n_minus_1",
        title="N-1 Cut Acceptance Measured in SR1 AmBe",
    )
    _ambe_s2_cut_eff = get_cut_eff(
        events_salt_matched_to_simu,
        all_cut_list=cut_list,
        n_bins=16,
        coord="s2_area",
        plot=True,
        indv_cut_type="n_minus_1",
        title="N-1 Cut Acceptance Measured in SR1 AmBe",
    )
    _ambe_z_cut_eff = get_cut_eff(
        events_salt_matched_to_simu,
        all_cut_list=cut_list,
        n_bins=16,
        coord="z",
        plot=True,
        indv_cut_type="n_minus_1",
        title="N-1 Cut Acceptance Measured in SR1 AmBe",
    )

    mask_simu_cut = apply_cut_lists(events_simu_matched_to_salt, cut_list)
    ambe_s1_cut_eff = get_cut_eff(
        events_salt_matched_to_simu[mask_simu_cut],
        all_cut_list=cut_list,
        n_bins=16,
        coord="s1_area",
        plot=True,
        indv_cut_type="n_minus_1",
        title="Conditional N-1 Cut Acceptance Measured in SR1 AmBe",
    )
    ambe_s2_cut_eff = get_cut_eff(
        events_salt_matched_to_simu[mask_simu_cut],
        all_cut_list=cut_list,
        n_bins=16,
        coord="s2_area",
        plot=True,
        indv_cut_type="n_minus_1",
        title="Conditional N-1 Cut Acceptance Measured in SR1 AmBe",
    )
    ambe_z_cut_eff = get_cut_eff(
        events_salt_matched_to_simu[mask_simu_cut],
        all_cut_list=cut_list,
        n_bins=16,
        coord="z",
        plot=True,
        indv_cut_type="n_minus_1",
        title="Conditional N-1 Cut Acceptance Measured in SR1 AmBe",
    )

    mask_salt_cut = apply_cut_lists(events_salt_matched_to_simu, cut_list)
    compare_bands(
        events_salt_matched_to_simu[mask_salt_cut],
        events_simu_matched_to_salt[mask_salt_cut],
        title="After Cuts Ambience Interference in SR1 AmBe HEV",
        coords=["z", "s2_range_50p_area"],
        n_bins=16,
    )
    compare_bands(
        events_salt_matched_to_simu[mask_salt_cut],
        events_simu_matched_to_salt[mask_salt_cut],
        title="After Cuts Ambience Interference in SR1 AmBe HEV",
        coords=["z", "s2_range_90p_area"],
        n_bins=16,
    )
    compare_bands(
        events_salt_matched_to_simu[mask_salt_cut],
        events_simu_matched_to_salt[mask_salt_cut],
        title="After Cuts Ambience Interference in SR1 AmBe HEV",
        coords=["s1_area", "s1_rise_time"],
        n_bins=16,
    )
    compare_bands(
        events_salt_matched_to_simu[mask_salt_cut],
        events_simu_matched_to_salt[mask_salt_cut],
        title="After Cuts Ambience Interference in SR1 AmBe HEV",
        coords=["s1_area", "s1_range_50p_area"],
        n_bins=16,
    )
    compare_bands(
        events_salt_matched_to_simu[mask_salt_cut],
        events_simu_matched_to_salt[mask_salt_cut],
        title="After Cuts Ambience Interference in SR1 AmBe HEV",
        coords=["s1_area", "s1_range_90p_area"],
        n_bins=16,
    )

    show_area_bias(
        events_salt_matched_to_simu[mask_salt_cut & mask_simu_cut],
        events_simu_matched_to_salt[mask_salt_cut & mask_simu_cut],
        title="After Cuts Ambience Interference in SR1 AmBe HEV",
        fraction=True,
        coord="s1_area",
        s1s2="s1",
        n_bins=16,
        ylim=(-10, 30),
    )
    show_area_bias(
        events_salt_matched_to_simu[mask_salt_cut & mask_simu_cut],
        events_simu_matched_to_salt[mask_salt_cut & mask_simu_cut],
        title="After Cuts Ambience Interference in SR1 AmBe HEV",
        fraction=True,
        coord="s2_area",
        s1s2="s2",
        n_bins=16,
        ylim=(-10, 30),
    )
    # show_area_bias(
    #    events_salt_matched_to_simu[mask_salt_cut & mask_simu_cut],
    #    events_simu_matched_to_salt[mask_salt_cut & mask_simu_cut],
    #    title="After Cuts Ambience Interference in SR1 AmBe HEV",
    #    coord="z",
    #    s1s2="s2",
    #    n_bins=16,
    #    ylim=(-2, 100)
    # )

## Event building efficiency

Assume we use match S1 time to account for event building efficiency, because it is closest to our NV tagging workflow. We need to account for the event loss in matching simulation to sprinkled, before any cuts. The loss comes from 
- `n_competing` event building loss
- Small physical S1/S2s beaten by uncorrelated S1/S2 from ambience, and the physical ones become `alt_s1`/`alt_s2`.
- Small S1s get merged with near by hits and get misidentified as S2

However, **keep in mind that this cannot reproduce the case when NV gamma coincides with unphysical S1 in TPC.**

In [ ]:
# Use the indices in inds_dict to do matching
events_salt_matched_to_simu = events_salt[inds_dict["ind_salt_s1_found"]]
events_simu_matched_to_salt = events_simu[inds_dict["ind_simu_s1_found"]]

# Apply cuts
mask_salt_cut = apply_cut_lists(events_salt_matched_to_simu, AmBe_CUTS_EXCEPT_S2Pattern)
mask_salt_min = apply_cut_lists(events_salt_matched_to_simu, ALL_CUTS_MINIMAL)
mask_simu_cut = apply_cut_lists(events_simu_matched_to_salt, AmBe_CUTS_EXCEPT_S2Pattern)
mask_simu_min = apply_cut_lists(events_salt_matched_to_simu, ALL_CUTS_MINIMAL)
mask_all_simu_min = apply_cut_lists(events_simu, ALL_CUTS_MINIMAL)
mask_all_simu_cut = apply_cut_lists(events_simu, AmBe_CUTS_EXCEPT_S2Pattern)

# Print out the stats
print("Statistics of sprinkled dataset before matching and cuts:", len(events_salt))
print("Statistics of simulated dataset before matching and cuts:", len(events_simu))
print("Statistics of sprinkled when matching S1s' time:", len(events_salt_matched_to_simu))
print(
    "Statistics of sprinkled after ALL_CUTS_MINIMAL cuts:",
    len(events_salt_matched_to_simu[mask_salt_min]),
)
print(
    "Statistics of sprinkled after AmBe_CUTS_EXCEPT_S2Pattern cuts:",
    len(events_salt_matched_to_simu[mask_salt_cut]),
)

For efficiency estimation below, we will try to use Clopper-Pearson uncertainty estimation

In [ ]:
saltax.show_eff1d(
    events_simu[mask_all_simu_min],  # All simulated events passing minimal cuts
    events_simu_matched_to_salt[mask_simu_min],  # Matched simulated events who passed minimal cuts
    mask_simu_cut[mask_simu_min],  # Mask for passing all cuts
    coord="e_ces",
    bins=np.linspace(0, 12, 25),
    labels_hist=[
        "All Simulation w ALL_CUTS_MINIMAL",
        "Matched Simulation w ALL_CUTS_MINIMAL",
        "Matched Simulation w AmBe_CUTS_EXCEPT_S2Pattern",
    ],
    labels_eff=[
        "Matching given ALL_CUTS_MINIMAL",
        "AmBe_CUTS_EXCEPT_S2Pattern given ALL_CUTS_MINIMAL + Matched",
    ],
    title="Matching Acceptance and Cut Acceptance",
)

In [ ]:
saltax.show_eff1d(
    events_simu[mask_all_simu_min],  # All simulated events passing minimal cuts
    events_simu_matched_to_salt[mask_simu_min],  # Matched simulated events who passed minimal cuts
    mask_simu_cut[mask_simu_min],  # Mask for passing all cuts
    coord="s1_area",
    bins=np.linspace(0, 100, 25),
    labels_hist=[
        "All Simulation w ALL_CUTS_MINIMAL",
        "Matched Simulation w ALL_CUTS_MINIMAL",
        "Matched Simulation w AmBe_CUTS_EXCEPT_S2Pattern",
    ],
    labels_eff=[
        "Matching given ALL_CUTS_MINIMAL",
        "AmBe_CUTS_EXCEPT_S2Pattern given ALL_CUTS_MINIMAL + Matched",
    ],
    title="Matching Acceptance and Cut Acceptance",
)

In [ ]:
saltax.show_eff1d(
    events_simu[mask_all_simu_min],  # All simulated events passing minimal cuts
    events_simu_matched_to_salt[mask_simu_min],  # Matched simulated events who passed minimal cuts
    mask_simu_cut[mask_simu_min],  # Mask for passing all cuts
    coord="s2_area",
    bins=np.linspace(200, 4500, 51),
    labels_hist=[
        "All Simulation w ALL_CUTS_MINIMAL",
        "Matched Simulation w ALL_CUTS_MINIMAL",
        "Matched Simulation w AmBe_CUTS_EXCEPT_S2Pattern",
    ],
    labels_eff=[
        "Matching given ALL_CUTS_MINIMAL",
        "AmBe_CUTS_EXCEPT_S2Pattern given ALL_CUTS_MINIMAL + Matched",
    ],
    title="Matching Acceptance and Cut Acceptance",
)

Note we compare the area spectrums with and without ambience interference, afer matching S1 timing + some cuts.

In [ ]:
mask_salt_tc2 = events_salt_matched_to_simu["s1_tight_coincidence"] == 2
mask_simu_tc2 = events_simu_matched_to_salt["s1_tight_coincidence"] == 2
mask_all_simu_tc2 = events_simu["s1_tight_coincidence"] == 2

plt.figure(dpi=150)
# Expect this as ambience interfered NV tagged data after minimal cuts (excluding 3-fold tc)
plt.hist(
    events_salt_matched_to_simu[mask_salt_min & mask_salt_tc2]["s1_area"],
    bins=np.linspace(0, 15, 31),
    alpha=0.5,
    density=True,
    color="tab:blue",
    label="Matched Sprinkled: %s counts"
    % (len(events_salt_matched_to_simu[mask_salt_min & mask_salt_tc2])),
)
# Expect this as ambience-free NV tagged data after minimal cuts (excluding 3-fold tc)
plt.hist(
    events_simu[mask_all_simu_min & mask_all_simu_tc2]["s1_area"],
    bins=np.linspace(0, 15, 31),
    alpha=0.5,
    density=True,
    color="tab:red",
    label="All Simulated: %s counts" % (len(events_simu[mask_all_simu_min & mask_all_simu_tc2])),
)
plt.legend()
plt.title("TC=2 + ALL_CUTS_MINIMAL")
plt.xticks(np.arange(0, 15.1, 2.5))
plt.xlabel("S1 Area [PE]")
plt.ylabel("Normalized Counts")
plt.show()

In [ ]:
plt.figure(dpi=150)

plt.scatter(
    events_simu[mask_all_simu_min]["s1_area"],
    events_simu[mask_all_simu_min]["cs2"],
    color="tab:orange",
    s=0.5,
    label="All Simulated",
)

plt.scatter(
    events_simu_matched_to_salt[mask_salt_min]["s1_area"],
    events_simu_matched_to_salt[mask_salt_min]["cs2"],
    color="tab:red",
    s=0.5,
    label="Matched Simulated",
)

plt.scatter(
    events_salt_matched_to_simu[mask_salt_min]["s1_area"],
    events_salt_matched_to_simu[mask_salt_min]["cs2"],
    color="tab:blue",
    s=0.5,
    label="Matched Sprinkled",
)

plt.xlabel("S1 [PE]")
plt.ylabel("cS2 [PE]")
plt.xlim(0, 100)
plt.ylim(100, 7000)
plt.yscale("log")
plt.legend()
plt.title("After ALL_CUTS_MINIMAL")
plt.show()

In [ ]:
plt.figure(dpi=150)

plt.scatter(
    events_simu[mask_all_simu_cut]["s1_area"],
    events_simu[mask_all_simu_cut]["cs2"],
    color="tab:orange",
    s=0.5,
    label="All Simulated",
)

plt.scatter(
    events_simu_matched_to_salt[mask_salt_cut]["s1_area"],
    events_simu_matched_to_salt[mask_salt_cut]["cs2"],
    color="tab:red",
    s=0.5,
    label="Matched Simulated",
)

plt.scatter(
    events_salt_matched_to_simu[mask_salt_cut]["s1_area"],
    events_salt_matched_to_simu[mask_salt_cut]["cs2"],
    color="tab:blue",
    s=0.5,
    label="Matched Sprinkled",
)

plt.xlabel("S1 [PE]")
plt.ylabel("cS2 [PE]")
plt.xlim(0, 100)
plt.ylim(100, 7000)
plt.yscale("log")
plt.legend()
plt.title("After AmBe_CUTS_EXCEPT_S2Pattern")
plt.show()

In [ ]:
plt.figure(dpi=150)
plt.hist(
    events_simu[mask_all_simu_min]["s1_area"],
    bins=np.linspace(0, 100, 50),
    color="tab:orange",
    label="All Simulated",
    histtype="step",
)
plt.hist(
    events_salt_matched_to_simu[mask_salt_min]["s1_area"],
    bins=np.linspace(0, 100, 50),
    color="tab:blue",
    label="Matched Sprinkled",
    histtype="step",
)
plt.hist(
    events_simu_matched_to_salt[mask_salt_min]["s1_area"],
    bins=np.linspace(0, 100, 50),
    color="tab:red",
    label="Matched Simulated",
    histtype="step",
)
plt.legend()
plt.xlabel("S1 [PE]")
plt.title("After ALL_CUTS_MINIMAL")

plt.figure(dpi=150)
plt.hist(
    events_simu[mask_all_simu_min]["s2_area"],
    bins=np.linspace(0, 6000, 100),
    color="tab:orange",
    label="All Simulated",
    histtype="step",
)
plt.hist(
    events_salt_matched_to_simu[mask_salt_min]["s2_area"],
    bins=np.linspace(0, 6000, 100),
    color="tab:blue",
    label="Matched Sprinkled",
    histtype="step",
)
plt.hist(
    events_simu_matched_to_salt[mask_salt_min]["s2_area"],
    bins=np.linspace(0, 6000, 100),
    color="tab:red",
    label="Matched Simulated",
    histtype="step",
)
plt.legend()
plt.xlabel("S2 [PE]")
plt.title("After ALL_CUTS_MINIMAL")
plt.show()

In [ ]:
plt.figure(dpi=150)
plt.hist(
    events_simu[mask_all_simu_cut]["s1_area"],
    bins=np.linspace(0, 100, 50),
    color="tab:orange",
    label="All Simulated",
    histtype="step",
)
plt.hist(
    events_salt_matched_to_simu[mask_salt_cut]["s1_area"],
    bins=np.linspace(0, 100, 50),
    color="tab:blue",
    label="Matched Sprinkled",
    histtype="step",
)
plt.hist(
    events_simu_matched_to_salt[mask_salt_cut]["s1_area"],
    bins=np.linspace(0, 100, 50),
    color="tab:red",
    label="Matched Simulated",
    histtype="step",
)
plt.legend()
plt.xlabel("S1 [PE]")
plt.title("After AmBe_CUTS_EXCEPT_S2Pattern")

plt.figure(dpi=150)
plt.hist(
    events_simu[mask_all_simu_cut]["s2_area"],
    bins=np.linspace(0, 6000, 100),
    color="tab:orange",
    label="All Simulated",
    histtype="step",
)
plt.hist(
    events_salt_matched_to_simu[mask_salt_cut]["s2_area"],
    bins=np.linspace(0, 6000, 100),
    color="tab:blue",
    label="Matched Sprinkled",
    histtype="step",
)
plt.hist(
    events_simu_matched_to_salt[mask_salt_cut]["s2_area"],
    bins=np.linspace(0, 6000, 100),
    color="tab:red",
    label="Matched Simulated",
    histtype="step",
)
plt.legend()
plt.xlabel("S2 [PE]")
plt.title("After AmBe_CUTS_EXCEPT_S2Pattern")
plt.show()

## Physical S1/S2 downgraded to alt_S1/alt_S2

In [ ]:
# Use the indices in inds_dict to do matching
# This matching means the simulated S1 became alternative S1; also apply minimal cuts on simu
mask_simu_s1_made_alt = np.zeros(len(events_simu), dtype=bool)
mask_simu_s1_made_alt[inds_dict["ind_simu_s1_made_alt"]] = True
events_simu_s1_made_alt = events_simu[mask_all_simu_min & mask_simu_s1_made_alt]
events_salt_s1_made_alt = events_salt[inds_dict["ind_salt_s1_made_alt"]]

# This matching means the simulated S2 became alternative S2; also apply minimal cuts on simu
mask_simu_s2_made_alt = np.zeros(len(events_simu), dtype=bool)
mask_simu_s2_made_alt[inds_dict["ind_simu_s2_made_alt"]] = True
events_simu_s2_made_alt = events_simu[mask_all_simu_min & mask_simu_s2_made_alt]
events_salt_s2_made_alt = events_salt[inds_dict["ind_salt_s2_made_alt"]]

In [ ]:
show_eff1d(
    events_simu[mask_all_simu_min],
    events_simu_s1_made_alt,
    coord="e_ces",
    bins=np.linspace(0, 12, 25),
    labels_hist=["Simulation before matching & cuts", "Simulation S1 downgraded"],
    labels_eff=["S1 downgraded"],
    title="S1 Downgraded After ALL_CUTS_MINIMAL",
)

In [ ]:
show_eff1d(
    events_simu[mask_all_simu_min],
    events_simu_s2_made_alt,
    coord="e_ces",
    bins=np.linspace(0, 12, 25),
    labels_hist=["Simulation before matching & cuts", "Simulation S2 downgraded"],
    labels_eff=["S2 downgraded"],
    title="S2 Downgraded After ALL_CUTS_MINIMAL",
)

## Lost or split

See Waveform Viewer section for example waveforms

- Lost: most likely failed `n_competing` cretieria in event building. 
- Split: most likely sprinkled on top of many data "S1" and "S2"

In [ ]:
# Use the indices in inds_dict to do matching
# The event was lost: no overlap in event time range
mask_simu_event_lost = np.zeros(len(events_simu), dtype=bool)
mask_simu_event_lost[inds_dict["ind_simu_event_lost"]] = True
events_simu_event_lost = events_simu[mask_all_simu_min & mask_simu_event_lost]

# The event was split: more than one event overlapped with the simulated event time range
mask_simu_event_split = np.zeros(len(events_simu), dtype=bool)
mask_simu_event_split[inds_dict["ind_simu_event_split"]] = True
events_simu_event_split = events_simu[mask_all_simu_min & mask_simu_event_split]

In [ ]:
show_eff1d(
    events_simu[mask_all_simu_min],
    events_simu_event_lost,
    coord="e_ces",
    bins=np.linspace(0, 12, 25),
    labels_hist=["Simulation before matching & cuts", "Simulation event lost"],
    labels_eff=["Event lost"],
    title="Event Lost After ALL_CUTS_MINIMAL",
)

In [ ]:
show_eff1d(
    events_simu[mask_all_simu_min],
    events_simu_event_split,
    coord="e_ces",
    bins=np.linspace(0, 12, 25),
    labels_hist=["Simulation before matching & cuts", "Simulation event split"],
    labels_eff=["Event split"],
    title="Event Split After ALL_CUTS_MINIMAL",
)

## Z Error

A fun thing to check is that the replaced main S1/S2 will give a nonsential reconstructed z. Let's see how often it happens.

In [ ]:
plt.figure(dpi=150)
plt.hist(events_salt_matched_to_simu["z"] - events_simu_matched_to_salt["z"], bins=100)
plt.xlabel("Sprinkled-Simulated z [cm]")
plt.ylabel("Counts")
plt.yscale("log")
plt.title("Artificial Displacement in z by Ambience Interference")
plt.show()

In [ ]:
plt.figure(dpi=150)
plt.scatter(events_simu_matched_to_salt["z"], events_salt_matched_to_simu["z"], s=0.1)
plt.xlabel("Simulated z [cm]")
plt.ylabel("Sprinkled z [cm]")
plt.title("Artificial Displacement in z by Ambience Interference")
plt.show()

## Match events time

It mimick doing AmBe calibration without NV tagging S1, but knowing when this event happen.

In [ ]:
events_salt_matched_to_simu = events_salt[inds_dict["ind_salt_event_found"]]
events_simu_matched_to_salt = events_simu[inds_dict["ind_simu_event_found"]]
print("Statistics when matching events' time:", len(events_salt_matched_to_simu))

mask_salt_cut = apply_cut_lists(events_salt_matched_to_simu, AmBe_CUTS_EXCEPT_S2Pattern)
print("Statistics after cuts:", len(events_salt_matched_to_simu[mask_salt_cut]))

In [ ]:
super_mega_analysis(events_salt_matched_to_simu, events_simu_matched_to_salt)

## Match S1 time

This should be of most use to us, because in data we require S1 timing coincidence with neutrno veto. Here we alredy considered a +/-100ns fuzzy window extension when matching S1.

In [ ]:
events_salt_matched_to_simu = events_salt[inds_dict["ind_salt_s1_found"]]
events_simu_matched_to_salt = events_simu[inds_dict["ind_simu_s1_found"]]
print("Statistics when matching S1s' time:", len(events_salt_matched_to_simu))

mask_salt_cut = apply_cut_lists(events_salt_matched_to_simu, AmBe_CUTS_EXCEPT_S2Pattern)
mask_salt_min = apply_cut_lists(events_salt_matched_to_simu, ALL_CUTS_MINIMAL)
print("Statistics after minimal cuts:", len(events_salt_matched_to_simu[mask_salt_min]))
print(
    "Statistics after AmBe_CUTS_EXCEPT_S2Pattern cuts:",
    len(events_salt_matched_to_simu[mask_salt_cut]),
)

In [ ]:
# Mimic what happen if we only use minimal cuts (no 3-fold tc though) + NV tagging
super_mega_analysis(
    events_salt_matched_to_simu, events_simu_matched_to_salt, cut_list=ALL_CUTS_MINIMAL
)

In [ ]:
# Mimic what happen if we only use all AmBe cuts except for S2pattern (not reliable for fuse) + NV tagging
super_mega_analysis(
    events_salt_matched_to_simu, events_simu_matched_to_salt, cut_list=AmBe_CUTS_EXCEPT_S2Pattern
)

## Match S2 time

This is just for fun. There is no physical motivation for it since we never know the true timing of S2 in any calibraiton.

In [ ]:
events_salt_matched_to_simu = events_salt[inds_dict["ind_salt_s2_found"]]
events_simu_matched_to_salt = events_simu[inds_dict["ind_simu_s2_found"]]
print("Statistics when matching S2s' time:", len(events_salt_matched_to_simu))
mask_salt_cut = apply_cut_lists(events_salt_matched_to_simu, AmBe_CUTS_EXCEPT_S2Pattern)
print("Statistics after cuts:", len(events_salt_matched_to_simu[mask_salt_cut]))

In [ ]:
# super_mega_analysis(events_salt_matched_to_simu, events_simu_matched_to_salt)

## Match both S1 and S2

This case specifically check that the physical S1s and S2s are NOT replaced by any junk in event building. It is not true in general especially in dirty ambience, but just for sanity check.

In [ ]:
events_salt_matched_to_simu = events_salt[
    np.intersect1d(inds_dict["ind_salt_s2_found"], inds_dict["ind_salt_s1_found"])
]
events_simu_matched_to_salt = events_simu[
    np.intersect1d(inds_dict["ind_simu_s2_found"], inds_dict["ind_simu_s1_found"])
]
print("Statistics when matching both S1s and S2s' time:", len(events_salt_matched_to_simu))
mask_salt_cut = apply_cut_lists(events_salt_matched_to_simu, AmBe_CUTS_EXCEPT_S2Pattern)
print("Statistics after cuts:", len(events_salt_matched_to_simu[mask_salt_cut]))

In [ ]:
# super_mega_analysis(events_salt_matched_to_simu, events_simu_matched_to_salt)

# Waveform Viewer

Still we use the S1 timing matching to mimic AmBe NV selection. We will only care about run `051906` because I have only copied that one's `peaks` to Midway3.

## Watch some random things

In [ ]:
run_id = "051906"

In [ ]:
st_data = cutax.xenonnt_offline(xedocs_version=xedocs_version)

In [ ]:
(events_simu, events_salt, inds_dict) = saltax.load_events(  # index for matching
    [run_id], st_salt, st_simu,
    plugins=("event_info", "cuts_basic"),
)

# Match based on S1 timing, with +/- 100ns fuzzy extension window
events_salt_matched_to_simu = events_salt[inds_dict["ind_salt_s1_found"]]
events_simu_matched_to_salt = events_simu[inds_dict["ind_simu_s1_found"]]

# Apply minimal cuts
mask_salt_min = apply_cut_lists(events_salt_matched_to_simu, ALL_CUTS_MINIMAL)
matched_salt_wcut = events_salt_matched_to_simu[mask_salt_min]
matched_simu_wcut = events_simu_matched_to_salt[mask_salt_min]

Just blindly watch some waveforms.

In [ ]:
saltax.plot_event_wf(
    ind=66,  # Put a random index here
    st_salt=st_salt,
    st_simu=st_simu,
    st_data=st_data,
    runid=run_id,
    events_simu=matched_simu_wcut,
    events_salt=matched_salt_wcut,
    event_ext_window_ns=1e6,
    s1_ext_window_samples=100,
    s2_ext_window_samples=500,
    ylim=(0, 5),
)

Now we want to see low energy range, whose S1 is less than 10PE.

In [ ]:
saltax.plot_event_wf(
    ind=60,  # Put a random index here
    st_salt=st_salt,
    st_simu=st_simu,
    st_data=st_data,
    runid=run_id,
    events_simu=matched_simu_wcut[
        matched_salt_wcut["s1_area"] < 10
    ],  # whose sprinkled S1 is less than 10PE
    events_salt=matched_salt_wcut[
        matched_salt_wcut["s1_area"] < 10
    ],  # whose sprinkled S1 is less than 10PE
    event_ext_window_ns=1e6,
    s1_ext_window_samples=100,
    s2_ext_window_samples=1000,
    ylim=(0, 2),
)

## Watch examples that S1/S2 downgraded to alternative S1/S2

In [ ]:
(events_simu, events_salt, inds_dict) = saltax.load_events(  # index for matching
    [run_id], st_salt, st_simu,
    plugins=("event_info", "cuts_basic"),
)

# Match based on S1 timing, with +/- 100ns fuzzy extension window
events_salt_matched_to_simu = events_salt[inds_dict["ind_salt_s1_found"]]
events_simu_matched_to_salt = events_simu[inds_dict["ind_simu_s1_found"]]

# Apply minimal cuts
mask_salt_min = apply_cut_lists(events_salt_matched_to_simu, ALL_CUTS_MINIMAL)
mask_all_simu_min = apply_cut_lists(events_simu, ALL_CUTS_MINIMAL)
matched_salt_wcut = events_salt_matched_to_simu[mask_salt_min]
matched_simu_wcut = events_simu_matched_to_salt[mask_salt_min]

# Use the indices in inds_dict to do matching
# This matching means the simulated S1 became alternative S1; also apply minimal cuts on simu
events_simu_s1_made_alt = events_simu[inds_dict["ind_simu_s1_made_alt"]]
events_salt_s1_made_alt = events_salt[inds_dict["ind_salt_s1_made_alt"]]

# This matching means the simulated S2 became alternative S2; also apply minimal cuts on simu
events_simu_s2_made_alt = events_simu[inds_dict["ind_simu_s2_made_alt"]]
events_salt_s2_made_alt = events_salt[inds_dict["ind_salt_s2_made_alt"]]

In [ ]:
saltax.plot_event_wf(
    ind=5,  # Put a random index here
    st_salt=st_salt,
    st_simu=st_simu,
    st_data=st_data,
    runid=run_id,
    events_simu=events_simu_s1_made_alt,
    events_salt=events_salt_s1_made_alt,
    event_ext_window_ns=2.4e6,
    s1_ext_window_samples=100,
    s2_ext_window_samples=500,
    ylim=(0, 5),
)

## Watch examples of event lost/split

In [ ]:
(events_simu, events_salt, inds_dict) = saltax.load_events(  # index for matching
    [run_id], st_salt, st_simu,
    plugins=("event_info", "cuts_basic"),
)

# Use the indices in inds_dict to find lost events and split events
events_simu_event_lost = events_simu[inds_dict["ind_simu_event_lost"]]
events_simu_event_split = events_simu[inds_dict["ind_simu_event_split"]]

In [ ]:
# Lost
saltax.plot_event_wf(
    ind=0,  # Put a random index here
    st_salt=st_salt,
    st_simu=st_simu,
    st_data=st_data,
    runid=run_id,
    events_simu=events_simu_event_lost,
    events_salt=None,  # Put None when there is no sensible corresponding sprinkled event to show
    event_ext_window_ns=2.4e6,
    s1_ext_window_samples=100,
    s2_ext_window_samples=500,
    ylim=(0, 5),
)

In [ ]:
# Split
saltax.plot_event_wf(
    ind=0,  # Put a random index here
    st_salt=st_salt,
    st_simu=st_simu,
    st_data=st_data,
    runid=run_id,
    events_simu=events_simu_event_split,
    events_salt=None,  # Put None when there is no sensible corresponding sprinkled event to show
    event_ext_window_ns=2.4e6,
    s1_ext_window_samples=100,
    s2_ext_window_samples=500,
    ylim=(0, 5),
)